In [1]:
import SimpleITK as sitk
import numpy as np
import glob
import os

In [2]:
# 对医疗图像进行重采样，仅仅需要将out_spacing替换成自己想要的输出即可
def resample_image(itk_image, out_spacing=None):
    if out_spacing is None:
        out_spacing = [1.0, 1.0, 1.0]
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    # 根据输出out_spacing设置新的size
    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / out_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / out_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / out_spacing[2]))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

def resample_mask(itk_mask, out_spacing=None):
    if out_spacing is None:
        out_spacing = [1.0, 1.0, 1.0]
    original_spacing = itk_mask.GetSpacing()
    original_size = itk_mask.GetSize()

    # 根据输出out_spacing设置新的size
    out_size = [
        int(np.round(original_size[0] * original_spacing[0] / out_spacing[0])),
        int(np.round(original_size[1] * original_spacing[1] / out_spacing[1])),
        int(np.round(original_size[2] * original_spacing[2] / out_spacing[2]))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_mask.GetDirection())
    resample.SetOutputOrigin(itk_mask.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_mask.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkNearestNeighbor)

    return resample.Execute(itk_mask)

#z-score
sitk_NIF = sitk.NormalizeImageFilter ()

In [3]:
# imagepath = 'D:/HCC_VETC/VETC/image/foyi/include/*'
# maskpath = 'D:/HCC_VETC/VETC/segmentation/foyi/include/*'
# outputimagepath = 'E:/preprocessing/foyi/0/imageout'
# outputmaskpath = 'E:/preprocessing/foyi/0/maskout'


imagepath = 'E:/xinjie/STAS/STAS(+)nrrd/*'
maskpath = 'E:/xinjie/STAS/STAS(+)ROInrrd/*'
outputimagepath = 'E:/xinjie/STAS/STAS(+)resample'
outputmaskpath = 'E:/xinjie/STAS/STAS(+)ROIresample'


# #下面为出错的病人
# mask2path = 'E:/hwr/image/*'
# outputmask2path = 'E:/ssy'

In [10]:
#图像进行resampling+zscore
for fileimage in glob.glob(imagepath):    
    itk_image=sitk.ReadImage(fileimage)
    resampleimage=resample_image(itk_image)
    resample_zscore_image = sitk_NIF.Execute(resampleimage)
     #接下来输出文件名改为原名+resamplezscore
    (filepath, tempfilename) = os.path.split(fileimage)
    (filename, extension) = os.path.splitext(tempfilename)
    tempfilename = filename + "_resamplezscore" + extension
    file_out = os.path.join(outputimagepath, tempfilename)
    sitk.WriteImage(resample_zscore_image, file_out)

In [4]:
#mask进行resampling
for filemask in glob.glob(maskpath): 
    itk_mask=sitk.ReadImage(filemask)
    resamplemask=resample_mask(itk_mask)
     #接下来输出文件名改为原名+resample
    (filepath, tempfilename) = os.path.split(filemask)
    (filename, extension) = os.path.splitext(tempfilename)
    tempfilename = filename + "_resample" + extension
    file_out = os.path.join(outputmaskpath, tempfilename)
    sitk.WriteImage(resamplemask, file_out)

In [6]:
# #mask进行resampling
# for filemask in glob.glob(maskpath): 
#     itk_mask=sitk.ReadImage(filemask)
#     resamplemask=resample_mask(itk_mask)
#      #接下来输出文件名改为原名+resample
#     (filepath, tempfilename) = os.path.split(filemask)
#     (filename, extension) = os.path.splitext(tempfilename)
#     tempfilename = filename + "_resample" + extension
#     file_out = os.path.join(outputmask2path, tempfilename)
#     sitk.WriteImage(resamplemask, file_out)

RuntimeError: Exception thrown in SimpleITK ImageFileReader_Execute: D:\a\1\sitk-build\ITK\Modules\IO\NRRD\src\itkNrrdImageIO.cxx:292:
itk::ERROR: NrrdImageIO(00000164723CD8D0): ReadImageInformation: Error reading D:/HCC_VETC/VETC/segmentation/foyi/填补缺失值\huangxinxi_20min_Segmentation-tumor-label.nrrd:
[nrrd] nrrdLoad: fopen("D:/HCC_VETC/VETC/segmentation/foyi/\huangxinxi_20min_Segmentation-tumor-label.nrrd","rb") failed: No such file or directory
